# Kaggle Playground - Season 4 Episode 
## Binary Classification of Insurance Cross Selling

Competion link - https://www.kaggle.com/competitions/playground-series-s4e8

### Steps
- Import the necessary libraries, packages and modules
- Read the datsets as data framers

### Understand the problem

- class is the target variable
- It determines the class of a mushroom depending on the given variables

In [32]:
!pip install keras_tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ------ --------------------------------- 20.5/129.1 kB ? eta -:--:--
   ------ --------------------------------- 20.5/129.1 kB ? eta -:--:--
   ------------------------ -------------- 81.9/129.1 kB 651.6 kB/s eta 0:00:01
   ------------------------ -------------- 81.9/129.1 kB 651.6 kB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 447.0 kB/s eta 0:00:00


In [37]:
import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb
import logging
import matplotlib.pyplot as plt
import numpy as np
import optuna
import os
import pandas as pd
import pickle
import seaborn as sns
import tensorflow as tf
import time
import xgboost as xgb

from imblearn.over_sampling import RandomOverSampler
from optuna.samplers import TPESampler
from scipy.stats import randint, uniform
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, HistGradientBoostingClassifier)
from sklearn.feature_selection import SelectFromModel  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from tensorflow.keras import backend as K

sns.set()
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
start_time = time.time()

In [2]:
# Test to see if TensorFlow can utilize the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# Check if TensorFlow can access the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    print(f"TensorFlow can access the GPU: {physical_devices}")
else:
    print("TensorFlow is not detecting the GPU.")

TensorFlow can access the GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Load data
train_df = pd.read_csv('train.csv', index_col='id')
test_df = pd.read_csv('test.csv', index_col='id')

train_df.head()

logger.info(f"Train data load completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Train data load completed. Time elapsed: 38.73 seconds


In [5]:
test_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a


In [6]:
# # Checking for data imbalance in training dataframe

# temp = train_df['class'].value_counts()    # Counts the number of ones and zeroes in the column
# temp_df = pd.DataFrame({'class' : temp.index, 'values' : temp.values})    # Creating a data frame with value and their counts
# plt.figure(figsize = (4, 3))
# sns.barplot(x = 'class', y = 'values', data = temp_df)
# plt.show()

In [7]:
# plt.pie(temp_df['values'], labels = [0, 1])
# plt.show()

### Identify the target variable and features

- class is the target variable
- It determines the class of a mushroom depending on the given variables

### Remove duplicate rows

- Checked the sum of duplicated rows in train and test datasets
- No dupllicated rows in train dataframe

In [8]:
# train_df.duplicated().sum()

In [9]:
# test_df.duplicated().sum()

### Handling missing values

- Checked the missing values in column
- There are considerable amount of missing values in many columns

In [10]:
train_df.shape

(3116945, 21)

In [11]:
test_df.shape

(2077964, 20)

In [12]:
# train_df.info()

In [13]:
# test_df.info()

In [14]:
# Split the dataset into training and validation sets
raw_train_df, validation_df = train_test_split(train_df, train_size=0.75, random_state=1, stratify=train_df['class'])
logger.info(f"Train test split completed. Time elapsed: {time.time() - start_time:.2f} seconds")

raw_train_df.head(2)

INFO:__main__:Train test split completed. Time elapsed: 63.51 seconds


,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
748879,p,2.45,x,g,n,f,a,NaN,p,7.12,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
589395,p,2.71,b,NaN,k,f,a,NaN,k,7.78,...,NaN,NaN,g,NaN,w,f,f,k,g,u


In [15]:
raw_train_df.isnull().sum()

class                         0
cap-diameter                  3
cap-shape                    30
cap-surface              503295
cap-color                    10
does-bruise-or-bleed          6
gill-attachment          392685
gill-spacing             943507
gill-color                   49
stem-height                   0
stem-width                    0
stem-root               2067772
stem-surface            1486230
stem-color                   27
veil-type               2218034
veil-color              2055885
has-ring                     16
ring-type                 96564
spore-print-color       2137374
habitat                      36
season                        0
dtype: int64

In [16]:
test_df.isnull().sum()

cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [17]:
# Check for missing values

missing_values = raw_train_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage = (missing_values / len(raw_train_df)) * 100
missing_percentage

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.529421
cap-color                0.000428
does-bruise-or-bleed     0.000257
gill-attachment         16.797864
gill-spacing            40.360344
gill-color               0.002096
stem-height              0.000000
stem-width               0.000000
stem-root               88.452963
stem-surface            63.576375
stem-color               0.001155
veil-type               94.880712
veil-color              87.944474
has-ring                 0.000684
ring-type                4.130713
spore-print-color       91.430324
habitat                  0.001540
season                   0.000000
dtype: float64

In [18]:
# Check for missing values

missing_values = validation_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage = (missing_values / len(validation_df)) * 100
missing_percentage

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.524645
cap-color                0.000257
does-bruise-or-bleed     0.000257
gill-attachment         16.843528
gill-spacing            40.414919
gill-color               0.001027
stem-height              0.000000
stem-width               0.000000
stem-root               88.452037
stem-surface            63.476324
stem-color               0.001412
veil-type               94.895263
veil-color              87.914460
has-ring                 0.001027
ring-type                4.147134
spore-print-color       91.410957
habitat                  0.001155
season                   0.000000
dtype: float64

In [19]:
# Check for missing values

missing_values_test = test_df.isnull().sum()

# Calculate the percentage of missing values

missing_percentage_test = (missing_values_test / len(test_df)) * 100
missing_percentage_test

cap-diameter             0.000337
cap-shape                0.001492
cap-surface             21.506821
cap-color                0.000626
does-bruise-or-bleed     0.000481
gill-attachment         16.834796
gill-spacing            40.404694
gill-color               0.002358
stem-height              0.000048
stem-width               0.000000
stem-root               88.452543
stem-surface            63.595327
stem-color               0.001011
veil-type               94.878689
veil-color              87.880445
has-ring                 0.000914
ring-type                4.148051
spore-print-color       91.417224
habitat                  0.001203
season                   0.000000
dtype: float64

### Data Preprocessing
- Droping columns with more than 50% missing values
- Using simple imputer
- Encoding Categorical Variables

In [20]:
# Drop columns with extremely high missing values
columns_to_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color', 'stem-surface']
raw_train_df.drop(columns=columns_to_drop, inplace=True)
validation_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# # Check for missing values

# missing_values = train_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage = (missing_values / len(train_df)) * 100
# missing_percentage

In [21]:
# # Check for missing values

# missing_values_test = test_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df)) * 100
# missing_percentage_test

In [22]:
# Split dependent and independent variables

y_train = raw_train_df['class']
raw_train_df = raw_train_df.drop('class', axis=1)
y_val = validation_df['class']
validation_df = validation_df.drop('class', axis=1)

raw_train_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
748879,2.45,x,g,n,f,a,NaN,p,7.12,2.30,w,f,f,d,u
589395,2.71,b,NaN,k,f,a,NaN,k,7.78,2.33,g,f,f,g,u


In [23]:
validation_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
1706883,1.40,x,g,y,f,d,d,n,1.56,2.43,n,f,f,d,a
1554994,5.35,f,h,g,f,s,d,w,4.75,9.63,g,f,f,d,u


In [24]:
# Identify numerical and categorical columns
numerical_cols = raw_train_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = raw_train_df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print("Unique Value is each categorical column :")
    print(col, raw_train_df[col].nunique())

Unique Value is each categorical column :
cap-shape 61
Unique Value is each categorical column :
cap-surface 70
Unique Value is each categorical column :
cap-color 60
Unique Value is each categorical column :
does-bruise-or-bleed 24
Unique Value is each categorical column :
gill-attachment 69
Unique Value is each categorical column :
gill-spacing 44
Unique Value is each categorical column :
gill-color 54
Unique Value is each categorical column :
stem-color 47
Unique Value is each categorical column :
has-ring 21
Unique Value is each categorical column :
ring-type 33
Unique Value is each categorical column :
habitat 48
Unique Value is each categorical column :
season 4


In [25]:
# Define the imputer for numerical columns (median imputation)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define a function to apply LabelEncoder to each categorical column
def encode_labels(df, columns):
    df_encoded = df.copy()
    le_dict = {}
    for col in columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        le_dict[col] = le
    return df_encoded, le_dict

# Define the encoder for categorical columns using LabelEncoder
def encode_labels_pipeline(X):
    df = pd.DataFrame(X, columns=categorical_cols)
    df_encoded, le_dict = encode_labels(df, categorical_cols)
    return df_encoded.values

# Create a ColumnTransformer for numerical and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', FunctionTransformer(func=lambda X: encode_labels_pipeline(X), validate=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Fit the preprocessor on the training data
preprocessor.fit(raw_train_df)

# Transform both the training, validation, and test data
train_df_transformed = preprocessor.transform(raw_train_df)
validation_df_transformed = preprocessor.transform(validation_df)
test_df_transformed = preprocessor.transform(test_df)

print("Transformed Training Data Shape:", train_df_transformed.shape)
print("Transformed Validation Data Shape:", validation_df_transformed.shape)
print("Transformed Test Data Shape:", test_df_transformed.shape)

logger.info(f"Missing values and categorical columns treatment completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Missing values and categorical columns treatment completed. Time elapsed: 234.51 seconds


Transformed Training Data Shape: (2337708, 15)
Transformed Validation Data Shape: (779237, 15)
Transformed Test Data Shape: (2077964, 15)


In [26]:
# Combine feature names
all_feature_names = numerical_cols.tolist() + categorical_cols.tolist()

# Convert to DataFrame
train_df_transformed = pd.DataFrame(train_df_transformed, columns=all_feature_names)
validation_df_transformed = pd.DataFrame(validation_df_transformed, columns=all_feature_names)
test_df_transformed = pd.DataFrame(test_df_transformed, columns=all_feature_names)

# # Check for missing values
# missing_values_train = train_df_transformed.isnull().sum()

# # Calculate the percentage of missing values
# missing_values_train = (missing_values_train / len(train_df_transformed)) * 100
# missing_values_train

In [27]:
# # Check for missing values

# missing_values_test = test_df_transformed.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df_transformed)) * 100
# missing_percentage_test

In [28]:
train_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-0.829222,0.285999,-1.093682,58.0,46.0,45.0,8.0,37.0,44.0,39.0,43.0,3.0,12.0,21.0,2.0
1,-0.773366,0.530507,-1.089977,36.0,70.0,42.0,8.0,37.0,44.0,34.0,27.0,3.0,12.0,25.0,2.0


In [29]:
validation_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-1.054794,-1.773796,-1.077626,35.0,16.0,41.0,3.0,10.0,7.0,23.0,24.0,5.0,12.0,7.0,0.0
1,-0.206215,-0.592007,-0.188388,21.0,17.0,25.0,3.0,24.0,7.0,32.0,17.0,5.0,12.0,7.0,2.0


In [30]:
test_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,0.500575,1.771571,0.736668,59.0,59.0,44.0,18.0,66.0,35.0,52.0,51.0,17.0,15.0,16.0,0.0
1,0.126771,-1.881231,-0.050062,50.0,53.0,45.0,5.0,66.0,17.0,54.0,38.0,6.0,14.0,16.0,0.0


In [41]:
# Encode the target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

y_val

array([1, 0, 0, ..., 0, 1, 0])

In [34]:
def matthews_correlation_coefficient(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred > 0.5, 'float32')  # Assuming binary classification with a 0.5 threshold

    tp = K.sum(y_true * y_pred)
    tn = K.sum((1 - y_true) * (1 - y_pred))
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    numerator = tp * tn - fp * fn
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + K.epsilon())

    return numerator / denominator

In [35]:
def build_model(hp):
    hp_seq_model = Sequential()
    
    hp_seq_model.add(Dense(units=hp.Int('units_input', min_value=8, max_value=128, step=8),
                           activation=hp.Choice('activation_input', values=['relu', 'tanh', 'sigmoid']),
                           input_dim=train_df_transformed.shape[1]))
    
    for i in range(hp.Int('no_hid_layer', min_value=1, max_value=10)):
        hp_seq_model.add(Dense(units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                              activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid'])))
        hp_seq_model.add(Dropout(rate=hp.Choice('dropout_' + str(i), values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5])))
        
    hp_seq_model.add(Dense(units=1, activation='sigmoid'))
    
    hp_seq_model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd']),
                        loss='binary_crossentropy',
                        metrics=[matthews_correlation_coefficient])
    
    return hp_seq_model

In [42]:
tuner = kt.RandomSearch(build_model,
                       objective=kt.Objective('val_matthews_correlation_coefficient', direction='max'),
                       max_trials=20,
                       executions_per_trial=3,
                       directory='my_dir',
                       project_name='Hyperparameter_Optimization')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, verbose=1)

batch_sizes = [32, 64, 128]
results = {}

for batch_size in batch_sizes:
    print(f"Testing batch size : {batch_size}")
    tuner.search(train_df_transformed, y_train,
                epochs=50,
                validation_data=(validation_df_transformed, y_val),
                callbacks=[early_stopping],
                batch_size=batch_size)
    
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print("Best hyperparameters : ", best_hps.values)
    
    tuned_seq_model = tuner.get_best_models(num_models=1)[0]
    
    tuned_seq_history = tuned_seq_model.fit(train_df_transformed, y_train,
                                            epochs=100, 
                                            validation_data=(validation_df_transformed, y_val), 
                                            callbacks=[early_stopping], 
                                            batch_size=batch_size)
    
    scores = tuned_seq_model.evaluate(validation_df_transformed, y_val, verbose=0)
    mcc = scores[1]  # MCC as per the compile metrics order
    
    results[batch_size] = {'model': tuned_seq_model, 'history': tuned_seq_history.history, 'mcc': mcc}
    
# Evaluate the performance of different batch sizes
for batch_size, result in results.items():
    print(f"Batch size: {batch_size} - MCC: {result['mcc']}")
        
# Choose the best batch size based on MCC
best_batch_size = max(results, key=lambda x: results[x]['mcc'])
print(f"The best batch size is: {best_batch_size}")

Reloading Tuner from my_dir\Hyperparameter_Optimization\tuner0.json
Testing batch size : 32

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
64                |64                |units_input
relu              |relu              |activation_input
9                 |9                 |no_hid_layer
128               |128               |units_0
tanh              |tanh              |activation_0
0                 |0                 |dropout_0
adam              |adam              |optimizer
32                |32                |units_1
sigmoid           |sigmoid           |activation_1
0.1               |0.1               |dropout_1
56                |56                |units_2
tanh              |tanh              |activation_2
0                 |0                 |dropout_2
96                |96                |units_3
tanh              |tanh              |activation_3
0.4               |0.4               |dropout_3
8                 |8                 |units_

KeyboardInterrupt: 